In [1]:
import numpy as np
import mne
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import utils

In [2]:
# List of file paths for all .set files you want to use as the training set
file_paths = [
    'D:\CJ AI Internship\EEG_Result\EEG_Result\\Grip_Both_01\\Grip_Both_01.set',
    'D:\CJ AI Internship\EEG_Result\EEG_Result\\Grip_Both_02\\Grip_Both_02.set',
    'D:\CJ AI Internship\EEG_Result\EEG_Result\\Grip_Both_03\\Grip_Both_03.set',
    'D:\CJ AI Internship\EEG_Result\EEG_Result\\Grip_Both_04\\Grip_Both_04.set',
    'D:\CJ AI Internship\EEG_Result\EEG_Result\\Grip_Both_05\\Grip_Both_05.set'
]

In [3]:
# Initialize empty lists to store data and labels
train_data = []
train_labels = []

In [4]:
# Loop through each file
for file_path in file_paths:
    # Load the preprocessed EEG data from each file
    epoch_data = mne.read_epochs_eeglab(file_path)
    data = epoch_data.get_data()

    # Assuming event occurs in the first 5 epochs (0 to 4) and skip every other epoch
    event_epochs = [0, 1, 2, 3, 4]

    # Concatenate the required time points (1 second before and 2 seconds after event)
    epochs_concatenated = np.concatenate([data[i] for i in event_epochs], axis=1)

    # Append the data and labels to the training lists
    train_data.append(epochs_concatenated)
    train_labels.append([1] * len(event_epochs))


FileNotFoundError: File does not exist: D:\CJ AI Internship\EEG_Result\EEG_Result\Grip_Both_01\Grip_Both_01.set

In [ ]:
# Normalize the data
train_data = np.array(train_data)
train_data = (train_data - np.mean(train_data)) / np.std(train_data)


# Convert the labels to numpy array
train_labels = np.array(train_labels).flatten()

In [ ]:
# Print the shapes to verify the number of samples
print(train_data.shape)    # Should be (25, 14, 384)
print(train_labels.shape)  # Should be (25,)

In [ ]:
# Create and train the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(14, 384, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Reshape train_data for the CNN model
train_data = train_data.reshape((-1, 14, 384, 1))

In [ ]:
# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=1, validation_split=0.2)

In [ ]:
# Step 1: Load and preprocess the test data
test_file_path = 'C:\\Users\\User\\Desktop\\EEG_Result\\Pinch_Both_05\\Pinch_Both_05.set'

# Load the preprocessed EEG data for the test file
test_epoch_data = mne.read_epochs_eeglab(test_file_path)
test_data = test_epoch_data.get_data()

# Assuming event occurs in these epochs in the test data
event_epochs_test = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Concatenate the required time points (1 second before and 2 seconds after event)
test_data_concatenated = np.concatenate([test_data[i] for i in event_epochs_test], axis=1)

# Normalize the test data using the same mean and standard deviation as the training data
test_data_normalized = (test_data_concatenated - np.mean(train_data)) / np.std(train_data)

# Reshape the test data for the CNN model
test_data_reshaped = test_data_normalized.reshape((-1, 14, 384, 1))

# Step 2: Evaluate the model on the test data
# Make predictions on the test data
predictions = model.predict(test_data_reshaped)

# Convert predictions to binary labels (event: 1, non-event: 0)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Create true labels for the test data (event: 1, non-event: 0)
true_labels = np.zeros(test_data_reshaped.shape[0])
true_labels[event_epochs_test] = 1

# Compare predicted labels with true labels to evaluate model performance
accuracy = np.mean(predicted_labels == true_labels)
print("Test accuracy:", accuracy)

